In [1]:
import sqlite3
import pandas as pd
from Bio import SeqIO
import os
import random

In [2]:
# Import Sheet
df = pd.read_csv("test_data/Testing_SQLite/samplesheet_nexus.csv")
df.rename(columns={"path": "read1"}, inplace=True)

In [3]:
df.head()

,sample,read1,reference_genome,sequencing_type
0,NIFP2402605,/90daydata/nece/bird/NECE_NEXUS_2025/Data/NIFP...,/90daydata/nece/bird/NECE_NEXUS_2025/Data/GCF_...,META_ONT_DNA
1,NIFP2402606,/90daydata/nece/bird/NECE_NEXUS_2025/Data/NIFP...,/90daydata/nece/bird/NECE_NEXUS_2025/Data/GCF_...,META_ONT_DNA
2,NIFP2402607,/90daydata/nece/bird/NECE_NEXUS_2025/Data/NIFP...,/90daydata/nece/bird/NECE_NEXUS_2025/Data/GCF_...,META_ONT_DNA
3,NIFP2402608,/90daydata/nece/bird/NECE_NEXUS_2025/Data/NIFP...,/90daydata/nece/bird/NECE_NEXUS_2025/Data/GCF_...,META_ONT_DNA
4,NIFP2402609,/90daydata/nece/bird/NECE_NEXUS_2025/Data/NIFP...,/90daydata/nece/bird/NECE_NEXUS_2025/Data/GCF_...,META_ONT_DNA


In [4]:
# Connect to (or create) the SQLite database
conn = sqlite3.connect("Testing.db")

In [5]:
#Creating Metadata table
conn.execute("""
CREATE TABLE IF NOT EXISTS metadata (
    sample TEXT,
    read1 TEXT,
    read2 TEXT,
    reference_genome TEXT,
    sequencing_type TEXT,
    location_name TEXT,
    location_coordinates TEXT,
    collection_date TEXT,
    sequencing_data TEXT,
    other_sample_information TEXT,
    long_term_storage_r1 TEXT,
    long_term_storage_r2 TEXT,
    PRIMARY KEY (sample, sequencing_type)
);
""")

In [6]:
#Columns that could be in the metadata
optional_fields = [
    "read2",
    "location_name",
    "location_coordinates",
    "collection_date",
    "sequencing_data",
    "other_sample_information",
    "long_term_storage_r1",
    "long_term_storage_r2"
]

#Colums that every sample should have
required_fields = ["sample", "read1", "reference_genome", "sequencing_type"]

#Check for required Colums
for field in required_fields:
    if field not in df.columns:
        raise ValueError(f"Missing required column: {field}")

# Add missing optional columns with None
for field in optional_fields:
    if field not in df.columns:
        df[field] = None

In [7]:
# Connect to database
cursor = conn.cursor()
# Attempt to insert each row individually and throw error if duplicate found
try:
    for row in df.itertuples(index=False):
        cursor.execute("""
            INSERT INTO metadata (
                sample, read1, reference_genome, sequencing_type,
                location_name, location_coordinates,
                collection_date, sequencing_data, other_sample_information
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            row.sample,
            row.read1,
            row.reference_genome,
            row.sequencing_type,
            row.location_name,
            row.location_coordinates,
            row.collection_date,
            row.sequencing_data,
            row.other_sample_information
        ))
    conn.commit()
except sqlite3.IntegrityError as e:
    raise RuntimeError(f"❌ Duplicate entry detected: {e}")
finally:
    conn.close()

In [8]:
# Connect to your SQLite database
conn = sqlite3.connect("Testing.db")
cursor = conn.cursor()

In [9]:
# Query all unique samples
cursor.execute("SELECT DISTINCT sample FROM metadata;")
rows = cursor.fetchall()

# Print the sample names
for row in rows:
    print(row[0])

NIFP2402605
NIFP2402606
NIFP2402607
NIFP2402608
NIFP2402609
NIFP2402610
NIFP2402611
NIFP2402612


In [10]:
# Close the connection
conn.close()

In [11]:
fasta_folder = "/Users/edwardbird/Documents/GitHub/edwardbirdlab-tools/test_data/Testing_SQLite"
db_path = "Testing.db"

In [12]:
# Connect to database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [13]:
#aking a new table for the contigs
cursor.execute("""
CREATE TABLE IF NOT EXISTS contigs (
    sample TEXT,
    sequencing_type TEXT,
    contig_id TEXT,
    sequence TEXT,
    PRIMARY KEY (sample, sequencing_type, contig_id),
    FOREIGN KEY (sample, sequencing_type) REFERENCES metadata(sample, sequencing_type)
);
""")

In [14]:
# Making a function to get the sequencing type from the metadata we imported already
def get_sequencing_type(sample_name):
    result = cursor.execute("""
        SELECT sequencing_type FROM metadata
        WHERE sample = ?
    """, (sample_name,)).fetchall()

    if len(result) == 1:
        return result[0][0]
    elif len(result) > 1:
        raise ValueError(f"Multiple sequencing types found for sample {sample_name}")
    else:
        raise ValueError(f"No metadata entry found for sample {sample_name}")

In [15]:
# Go though fasta files and import them
for fname in os.listdir(fasta_folder):
    if fname.endswith(".fasta") or fname.endswith(".fa"):
        sample = os.path.splitext(fname)[0].split("_")[0]
        try:
            sequencing_type = get_sequencing_type(sample)
        except ValueError as e:
            print(f"Skipping {fname}: {e}")
            continue

        fasta_path = os.path.join(fasta_folder, fname)
        # Counter for inserted sequences
        inserted_count = 0

        for record in SeqIO.parse(fasta_path, "fasta"):
            contig_id = record.id
            sequence = str(record.seq)

            try:
                cursor.execute("""
                    INSERT INTO contigs (sample, sequencing_type, contig_id, sequence)
                    VALUES (?, ?, ?, ?)
                """, (sample, sequencing_type, contig_id, sequence))
                inserted_count += 1
            except sqlite3.IntegrityError:
                print(f"Duplicate entry skipped: {sample}, {sequencing_type}, {contig_id}")

        # Print summary for this sample
        print(f"✅ Added {inserted_count} sequences for sample '{sample}' (type: {sequencing_type})")

✅ Added 19744 sequences for sample 'NIFP2402609' (type: META_ONT_DNA)
✅ Added 14230 sequences for sample 'NIFP2402606' (type: META_ONT_DNA)
✅ Added 21851 sequences for sample 'NIFP2402607' (type: META_ONT_DNA)
✅ Added 60455 sequences for sample 'NIFP2402612' (type: META_ONT_DNA)
✅ Added 71171 sequences for sample 'NIFP2402608' (type: META_ONT_DNA)
✅ Added 45472 sequences for sample 'NIFP2402605' (type: META_ONT_DNA)
✅ Added 22078 sequences for sample 'NIFP2402610' (type: META_ONT_DNA)
✅ Added 31189 sequences for sample 'NIFP2402611' (type: META_ONT_DNA)


In [16]:
conn.commit()
conn.close()

In [17]:
# Testing out grabing a random contig
# Connect to the database
conn = sqlite3.connect("Testing.db", timeout=10)
cursor = conn.cursor()

# Query to get all contig ids for the sample 'NIFP2402606'
cursor.execute("""
    SELECT contig_id, sequence FROM contigs
    WHERE sample = 'NIFP2402606';
""")

# Fetch all results
contigs = cursor.fetchall()

# If there are contigs, select a random one
if contigs:
    random_contig = random.choice(contigs)
    contig_id, sequence = random_contig
    print(f"Random contig ID: {contig_id}")
    print(f"Sequence: {sequence}")
else:
    print("No contigs found for sample NIFP2402606")

# Close the connection
conn.close()

Random contig ID: NoClass_5c2f68dc-c8ac-433e-8883-054337bb7960
Sequence: GGTATTCAACAGTAGGCCCCGCTCGACTGTAGCCTTAGCTTACTTGTTTTTGTAATACTTTCAATTTTTTAAATTTCCCCCGTAAAATTTCTTTGATCTCCAAGCAAATTGTGGTTTCTTTCAAATATTTTTCGAAAGGTGTAGCAGCTCACAAATATAAATATCATTCAAAAGGTGTTTTACATAAATGTTTGTTTTTCTAAGGATTTTCCGGCACATATAAATCTTATAAGAATATATTTTCCAGACTTTTAAATGATCTTTCCATACAAAAGTCATGGTACTGTTAAACATTTTTTTAAAGGTCTTTGTTTTTCTTTTTTTAGAAAATCTTTGGAATATGAACGCGATTTTTTCCGTATTCTTTAATTTTTTTTTTTTTCAACGAAATCCTAGCAGAAAATCGTGGATTTTGTAAAAAAACGTTTAAAAGTCTTGGATATTCCTAAAACATTCTATACACTTTAAGGGATCATACTCTCACTCAAGGATATTCTTGATTGAAAAATCTTGGTATTCTTGACAAAAATACTTTTTCAATCTTATTTTTTATAAATAAAATAGTGGTGGTGAACATTCACTGAAGGGGTTTTTTATTGAAAAGTGAGATTTATTTTTCTTAAAATTTTCCCACACATAAATATGTATTATACCACTGTACAAATTTGAACGCATCTTATTGTTCCCTAGCCGAAGCATATA
